In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.stats.api as sms
from scipy import stats


In [2]:
df = pd.read_excel('../comparing_ir.xlsx',usecols=[1,2])
df.index = pd.date_range(start='2010-06',end='2022-04',freq='M')
df.head()


,FED_Policy_Rate,BOE_Policy_Rate
2010-06-30,0.25,0.5
2010-07-31,0.25,0.5
2010-08-31,0.25,0.5
2010-09-30,0.25,0.5
2010-10-31,0.25,0.5


In [3]:
before_covid = df[(df.index <'2020-03')]
after_covid = df.iloc[112:142,:] 
print(f'n of post corona : {before_covid.shape}\nn of after corona : {after_covid.shape}')

n of post corona : (117, 2)
n of after corona : (30, 2)


In [4]:
model_post_corona,model_corona = sm.OLS(before_covid['BOE_Policy_Rate'],sm.add_constant(before_covid['FED_Policy_Rate'])),sm.OLS(after_covid['BOE_Policy_Rate'],sm.add_constant(after_covid['FED_Policy_Rate']))

results_post_corona,results_corona = model_post_corona.fit(),model_corona.fit()

results_post_corona.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        BOE_Policy_Rate   R-squared:                       0.271
Model:                            OLS   Adj. R-squared:                  0.265
Method:                 Least Squares   F-statistic:                     42.83
Date:                Tue, 17 May 2022   Prob (F-statistic):           1.73e-09
Time:                        15:09:28   Log-Likelihood:                 87.233
No. Observations:                 117   AIC:                            -170.5
Df Residuals:                     115   BIC:                            -164.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.4372      0.015     28.613      0.000       0.407       0.467
FED_Policy_Rate     0.0922      0.014      6.545      0.000       0.064       0.120
==============================================================================
Omnibus:                       36.760   Durbin-Watson:                   0.126
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               57.964
Skew:                          -1.569   Prob(JB):                     2.59e-13
Kurtosis:                       4.432   Cond. No.                         2.46
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [5]:
results_corona.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        BOE_Policy_Rate   R-squared:                       0.776
Model:                            OLS   Adj. R-squared:                  0.768
Method:                 Least Squares   F-statistic:                     96.83
Date:                Tue, 17 May 2022   Prob (F-statistic):           1.37e-10
Time:                        15:09:29   Log-Likelihood:                 20.396
No. Observations:                  30   AIC:                            -36.79
Df Residuals:                      28   BIC:                            -33.99
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.0503      0.031      1.603      0.120      -0.014       0.115
FED_Policy_Rate     0.4093      0.042      9.840      0.000       0.324       0.494
==============================================================================
Omnibus:                       40.715   Durbin-Watson:                   0.316
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              124.992
Skew:                           2.950   Prob(JB):                     7.22e-28
Kurtosis:                      11.073   Cond. No.                         2.40
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Normality Tests

In [6]:
print(f'JB Test P Values\n\n\nPost Corona : {stats.jarque_bera(results_post_corona.resid)[1]:.4f}\nAfter Corona : {stats.jarque_bera(results_corona.resid)[1]:.4f}')

JB Test P Values


Post Corona : 0.0000
After Corona : 0.0000


Heteroscedasticity Test

In [7]:
print(f'Breusch-Pagan P Values\n\n\nPost Corona : {sms.het_breuschpagan(results_post_corona.resid,results_post_corona.model.exog)[1]}\nAfter Corona : {sms.het_breuschpagan(results_corona.resid,results_corona.model.exog)[1]}')

Breusch-Pagan P Values


Post Corona : 0.006493633520132123
After Corona : 0.7148886986495444


Focusing On Post Corona

In [8]:
sms.acorr_breusch_godfrey(results_post_corona,nlags=1)[1]

3.8579391637238e-24

In [9]:
#Seems Serial Autocorrelation and Heteroscedasticity on BOE Post Corona OLS
#According to reason above we should switch GLS model

#Application of GLS
from scipy.linalg import toeplitz

res_endog = results_post_corona.resid[1:]
res_exog = results_post_corona.resid[:-1]
res_exog.index,res_endog.index = np.arange(116),np.arange(116)

res_fit = sm.OLS(res_endog,res_exog).fit()
rho = res_fit.params[0]
order = toeplitz(np.arange(before_covid.shape[0]))
sigma = rho**order


In [10]:
res_fit.pvalues

x1    3.179273e-54
dtype: float64

In [11]:
model_post_corona = sm.GLS(before_covid['BOE_Policy_Rate'],sm.add_constant(before_covid['FED_Policy_Rate']),sigma=sigma)
results_post_corona = model_post_corona.fit()

In [12]:
results_post_corona.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            GLS Regression Results                            
==============================================================================
Dep. Variable:        BOE_Policy_Rate   R-squared:                       0.006
Model:                            GLS   Adj. R-squared:                 -0.003
Method:                 Least Squares   F-statistic:                    0.6549
Date:                Tue, 17 May 2022   Prob (F-statistic):              0.420
Time:                        15:10:24   Log-Likelihood:                 210.05
No. Observations:                 117   AIC:                            -416.1
Df Residuals:                     115   BIC:                            -410.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.5074      0.067      7.620      0.000       0.376       0.639
FED_Policy_Rate     0.0327      0.040      0.809      0.420      -0.047       0.113
==============================================================================
Omnibus:                       99.920   Durbin-Watson:                   1.974
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5148.544
Skew:                          -2.105   Prob(JB):                         0.00
Kurtosis:                      35.224   Cond. No.                         2.14
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

FED has no sufficient affect on BOE's Policy Rates.Before Corona Period

In [13]:
# After Corona Serial Correlation Test

sms.acorr_breusch_godfrey(results_corona,nlags=1)[1]

2.1998136592491664e-06

In [14]:
sms.durbin_watson(results_corona.resid)

0.31551724046999274

In [24]:
#Building ARDL Model 
from statsmodels.tsa.api import ARDL

ardl = ARDL(after_covid['BOE_Policy_Rate'],1,sm.add_constant(after_covid['FED_Policy_Rate']),{'FED_Policy_Rate':1})
results_corona = ardl.fit()

c:\Users\acer\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\tsa\ardl\model.py:158: SpecificationWarning: exog contains variables that are missing from the order dictionary.  Missing keys: const.
  warnings.warn(msg, SpecificationWarning)


In [25]:
results_corona.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              ARDL Model Results                              
==============================================================================
Dep. Variable:        BOE_Policy_Rate   No. Observations:                   30
Model:                     ARDL(1, 1)   Log Likelihood                  40.310
Method:               Conditional MLE   S.D. of innovations              0.060
Date:                Tue, 17 May 2022   AIC                            -70.619
Time:                        15:12:09   BIC                            -63.783
Sample:                    11-30-2019   HQIC                           -68.478
                         - 03-31-2022                                         
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.0089      0.017      0.522      0.606      -0.026       0.044
BOE_Policy_Rate.L1     1.3373      0.148      9.049      0.000       1.033       1.642
FED_Policy_Rate.L0     0.3369      0.048      6.997      0.000       0.238       0.436
FED_Policy_Rate.L1    -0.4859      0.070     -6.955      0.000      -0.630      -0.342
======================================================================================
"""

In [26]:
results_corona.test_normality()

Jarque-Bera    5.252373e+01
P-value        3.932030e-12
Skewness       2.249777e-01
Kurtosis       9.577648e+00
dtype: float64

In [28]:
from sklearn.metrics import r2_score
endog = after_covid['FED_Policy_Rate'][1:]
r2_score(endog,results_corona.fittedvalues)

0.42069228824042404

In [29]:
results_corona.test_heteroskedasticity(1)['P-value']

Lag
1    0.766769
Name: P-value, dtype: float64

In [30]:
sms.durbin_watson(results_corona.resid)

2.2603725318069863

In [160]:
results_corona.test_serial_correlation(15)

,Ljung-Box,LB P-value,DF
Lag,,,
1,0.570265,NaN,0
2,2.914337,NaN,0
3,2.962521,NaN,0
4,2.962546,NaN,0
5,2.962562,0.085213,1
6,2.962668,0.227334,2
7,2.962818,0.397393,3
8,2.963025,0.564032,4
9,2.963299,0.705644,5
